In [1]:
# 경로설정
import sys
path = "C:\github/aide-analysis/"
sys.path.append(path)

import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd())))

# 한글폰트
import os
import matplotlib.pyplot as plt
if os.name =='posix':
    plt.rc('font',family = 'ApplGothic')
else:
    plt.rc('font',family = 'Malgun Gothic')
    
# 경고출력문 제거
import warnings
warnings.filterwarnings("ignore")

# 모듈 import`
import pandas as pd
import src
from src.openapi import KakaoLocalAPI as kakao
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from src.sql import DatabaseHandler
from src.temp import *
from datetime import datetime
import datetime
from dateutil.relativedelta import relativedelta
from src.bigquery import BigqueryHandler
import seaborn as sns
import numpy as np
import statistics
from tqdm import tqdm
import config
from core.config import Config
import json
from datetime import datetime
import geopandas as gpd
from tqdm import tqdm
import folium
from src.bigquery import bigquery_client, bigquery
import geopandas as gpd
from shapely import wkt
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl
config = Config()
BH = BigqueryHandler()


# DataFrame 디스플레이 설정
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

# 서울시 아파트(거래 20년이후)

In [2]:
sql =f"""
SELECT danji.complexNo, 
      danji.complexName,
      danji.sidoCode,
      danji.sigunguName,
      danji.bdongName,
      danji.realEstateTypeName,
      danji.address,
      danji.roadAddress,
      danji.totalHouseholdCount,
      danji.highFloor,
      danji.useApproveYear,
      danji.totalDongCount,
      danji.batlRatio,
      danji.btlRatio,
      area.jeonyongArea,
      area.jeonyongPyeong,
      area.supplyArea,
      area.supplyPyeong,
      area.roomCount,
      area.areaSixGroupNo,
      area.maxPrice,
      area.pyeongNo,
      price.tradeType,
      price.tradeYear,
      price.tradeMonth,
      price.contractDate,
      price.dealPrice,
      price.leasePrice,
      price.rentPrice,
      price.floor,
      danji.latitude,
      danji.longitude,
FROM `aidepartners.aide.complex_danji_information` AS danji
LEFT JOIN (SELECT complexNo, 
                  pyeongNo,
                  jeonyongArea,
                  jeonyongPyeong,
                  supplyArea,
                  supplyPyeong,
                  areaSixGroupNo,
                  maxPrice,
                  roomCount
           FROM `aidepartners.aide.complex_area_information_prod`
           ) AS area ON danji.complexNo = area.complexNo
LEFT JOIN (SELECT  complexNo,
                   areaNo,
                   tradeType,
                   tradeYear,
                   tradeMonth,
                   contractDate,
                   dealPrice,
                   leasePrice,
                   rentPrice,
                   floor,
                   deleteYn
           FROM `aidepartners.aide.complex_real_price`
           WHERE tradeYear>=2020
           ) AS price ON area.complexNo = price.complexNo 
                         AND area.pyeongNo = price.areaNo 
WHERE danji.sidoCode = '11' 
      AND danji.realEstateTypeName = '아파트' 
      AND price.deleteYn is null
"""
df = BH.read_table(sql)

In [3]:
df.shape

(731766, 32)

# To BigQuery

In [4]:
df = df.drop(columns = 'sidoCode')

In [5]:
# yearMonth 컬럼 생성
df['yearMonth'] = np.nan

idx = df[-df['contractDate'].isna()].index
df.loc[idx, 'yearMonth'] = df.loc[idx, 'tradeYear'].astype(str) + '-' +df.loc[idx,'tradeMonth'].astype(str)
df.loc[idx, 'yearMonth'] = pd.to_datetime(df.loc[idx,'yearMonth'],format = '%Y-%m')
df['yearMonth'] = pd.to_datetime(df['yearMonth'],format = '%Y-%m')

In [6]:

from src.bigquery import bigquery_client, bigquery
bq = bigquery_client


project = bq.project
dataset_ref = bigquery.DatasetReference(project, "viz")
new_table_id ="seoul_price_upper_2021_new"

table_ref = dataset_ref.table(new_table_id)
schema = [bigquery.SchemaField('complexNo',"STRING"),
          bigquery.SchemaField('complexName',"STRING"),
          bigquery.SchemaField('sidoName',"STRING"),
          bigquery.SchemaField('sigunguName',"STRING"),
          bigquery.SchemaField('bdongName',"STRING"),
          bigquery.SchemaField('realEstateTypeName',"STRING"),
          bigquery.SchemaField('address',"STRING"),
          bigquery.SchemaField('roadAddress',"STRING"),
          bigquery.SchemaField('totalHouseholdCount',"INTEGER"),
          bigquery.SchemaField('totalDongCount',"INTEGER"),
          bigquery.SchemaField('batlRatio',"FLOAT"),
          bigquery.SchemaField('btlRatio',"FLOAT"),
          bigquery.SchemaField('useApproveYear',"STRING"),
          bigquery.SchemaField('highFloor',"INTEGER"),
          bigquery.SchemaField('jeonyongArea',"FLOAT"),
          bigquery.SchemaField('jeonyongPyeong',"FLOAT"),
          bigquery.SchemaField('supplyArea',"FLOAT"),
          bigquery.SchemaField('supplyPyeong',"FLOAT"),
          bigquery.SchemaField('areaSixGroupNo',"STRING"),
          bigquery.SchemaField('maxPrice','INTEGER'),
          bigquery.SchemaField('pyeongNo',"STRING"),
          bigquery.SchemaField('roomCount',"INTEGER"),
          bigquery.SchemaField('tradeType',"STRING"),
          bigquery.SchemaField('tradeYear',"INTEGER"),
          bigquery.SchemaField('tradeMonth',"INTEGER"),
          bigquery.SchemaField('contractDate',"DATE"),
          bigquery.SchemaField('dealPrice',"INTEGER"),
          bigquery.SchemaField('leasePrice',"INTEGER"),
          bigquery.SchemaField('rentPrice',"INTEGER"),
          bigquery.SchemaField('floor',"INTEGER"),
          bigquery.SchemaField('latitude',"FLOAT"),
          bigquery.SchemaField('longitude',"FLOAT"),
          bigquery.SchemaField('yearMonth','DATE'),
         ]
table = bigquery.Table(table_ref, schema = schema)

# 테이블 최초 생성
table = bq.create_table(table)

job = bq.load_table_from_dataframe(df, table)
# job